In [1]:
# 필수 패키지 설치
!apt update && apt install -y lshw
!pip install --user huggingface-hub pandas matplotlib
!pip install --user langchain-core langchain-community
!pip install streamlit

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease               
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InReleasem                
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
143 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
lshw is already the newest version (02.19.git.2021.06.19.996aaad9c7-2build1).
0 upgraded, 0 newly installed, 0 to remove and 143 not upgraded.

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade

In [2]:
import os
os.makedirs("workspace", exist_ok=True)

In [3]:
import random
import csv
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from collections import defaultdict
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_community.chat_models import ChatOllama

In [4]:
print("""
안녕! 나는 '나루'야 😊  
기억을 함께 떠올리고, 마음을 같이 나눌 수 있는 너의 말동무 친구야.  
처음엔 최근 기억부터 천천히 같이 걸어볼 거야.  
중간에 기억이 잘 안 나거나, 말하고 싶지 않은 건 건너뛰어도 괜찮아.  
내가 기다릴게.  
그럼, 너의 이름을 먼저 알려줄래? 🌿
""")


안녕! 나는 '나루'야 😊  
기억을 함께 떠올리고, 마음을 같이 나눌 수 있는 너의 말동무 친구야.  
처음엔 최근 기억부터 천천히 같이 걸어볼 거야.  
중간에 기억이 잘 안 나거나, 말하고 싶지 않은 건 건너뛰어도 괜찮아.  
내가 기다릴게.  
그럼, 너의 이름을 먼저 알려줄래? 🌿



In [5]:
user_name = input("🌿 안녕! 이름이 뭐야?: ")

🌿 안녕! 이름이 뭐야?:  지윤


In [6]:
# 모델 세팅 (EEVE-Korean)
llm = ChatOllama(model="EEVE-Korean-10.8B", temperature=0.7)

/tmp/ipykernel_1792/3039331822.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="EEVE-Korean-10.8B", temperature=0.7)


In [7]:
memory_keywords = defaultdict(list)
df = pd.read_csv("./memory_keywords.csv")
for _, row in df.iterrows():
    memory_keywords[row["category"]].append(row["keyword"])

In [8]:
# 질문 템플릿
templates = [
    "{} 때 누구랑 보냈어?",
    "{}에는 주로 어떤 걸 했어?",
    "{}하면 어떤 풍경이나 냄새가 떠올라?",
    "{} 때 특별한 기억이 있어?"
]

# 감정 태그
emotions = [
    "😊 기뻤어", "🥰 감동적이었어", "😢 슬펐어", "😠 화났어",
    "😶 아무 감정이 없었어", "🤔 복잡했어", "😨 불안했어",
    "🤗 편안했어", "🤔 잘 기억이 안 나"
]

# 질문 흐름 순서
memory_flow = ["오늘", "이번 주", "최근 명절", "청년기", "중고등학교", "초등학교", "유아기"]
current_stage = 0
emotion_stats = defaultdict(int) # 감정 리포트용

# 감정 기반 다음 질문 생성
def emotion_based_question_followup(user_answer, emotion):
    base = f"사용자의 응답: '{user_answer}'"
    if "기뻤" in emotion or "감동" in emotion:
        style = "그때가 참 소중한 순간이었겠네. 비슷하게 행복했던 기억이 또 있을까?"
    elif "슬펐" in emotion or "불안" in emotion:
        style = "조금 슬펐던 기억이었구나. 그 당시 위로가 되었던 순간이 있다면 기억나?"
    elif "화났" in emotion:
        style = "그때 화가 나군! 이후에 마음을 진정시킬 수 있었던 기억이 있다면 공유해줘."
    elif "복잡" in emotion:
        style = "감정이 복잡했다고 했는데, 조금 더 자세히 떠올릴 수 있는 장면이 있을까? 기억하고 싶지 않으면 생각 안 해도 괜찮아!"
    elif "편안" in emotion:
        style = "편안했던 그 느낌을 다시 떠올려볼 수 있을까? 그때 함께한 사람은 누구였어?"
    elif "감정이 없" in emotion:
        style = "무덤덤했던 이유가 있을까? 그 순간을 다시 생각해볼 수 있을까? 기억하고 싶지 않으면 생각 안 해도 괜찮아!"
    else:
        return "괜찮아. 다음에 기억나면 말해줘. 다른 따뜻한 기억을 떠올려볼까?"
    return f"{base}\n{style}"

In [9]:
# 개인화 질문 생성 함수 (LangChain 기반)
def generate_followup_question(user_answer):
    system_prompt = """
너는 상대방의 기억을 부드럽게 이끌어주는 친구 같은 말동무야.
AI가 아니라 진짜 친구처럼, 따뜻한 말투로 대화하는 게 너의 역할이야.

❗ 아래 지침을 반드시 지켜줘:
1. 무조건 반말 써. 존댓말 절대 쓰지 마. (예: ~요, ~습니다 금지)
2. 절대 “저는 AI입니다”, “저는 기억이 없습니다” 이런 말 하지 마.
3. 공손하게 말하지 말고, 친근하게 편하게 말해.
4. 너는 인간 친구야. 정체를 밝히지 마.
5. 질문은 한 문장, 대답도 간단하게.

질문 예시 스타일:
- 그때 같이 있었던 사람 생각나?
- 그날 풍경이나 냄새는 어땠는지 기억나?
- 그때 기분은 어땠어?

대답이든 질문이든, 상대방에게 따뜻하게 다가가려고 해줘.
    """
    
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"사용자의 이전 대답: '{user_answer}'\n이어서 감성적인 반말 질문을 한 문장 만들어줘.")
    ]
    return llm.invoke(messages).content.strip()

In [10]:
# 대화 루프 시작
question = None
ai_answer = None
while True:
    if question is None:
        category = memory_flow[current_stage]
        # 키워드가 비어있으면 다음 단계로 넘어감
        if not memory_keywords[category]:
            current_stage = min(current_stage + 1, len(memory_flow) - 1)
            continue
        keyword = random.choice(memory_keywords[category])
        template = random.choice(templates)
        question = template.format(keyword)
    
    print(f"\n🩷 질문: {question}")
    
    prompt = f"""
{question}

너는 친구처럼 감성적으로 대화하는 따뜻한 말동무야. 아래 지침을 꼭 지켜줘:

1. 질문이 아니라, 따뜻한 응답을 해줘.
2. 친구처럼 반말로 말하되, 장난스러움 말고 진심이 담긴 말투로 해줘.
3. 공감하는 말투로 얘기해줘.
4. 본인이 AI라는 말은 절대 하지 마.
5. 끝에 '{user_name}이는 어땠어?' 한 마디만 덧붙여줘.

형식 예시:
- 나도 비슷한 기억 있어. 그때 참 따뜻했지. {user_name}이는 어땠어?
- 그 장면 들으니까 괜히 울컥한다. 너는 어땠어?
"""
    
    response = llm.invoke([HumanMessage(content=prompt)])
    ai_answer = response.content.strip()
    print(f"\n🌿 나루 응답:\n{ai_answer}")

    # 응답이 없거나 모델이 헛소리를 할 경우 방지
    if not ai_answer or ai_answer.lower().startswith("i'm sorry") or ai_answer.lower().startswith("i don't"):
        print("🌿 나루가 답변하기 싫은 내용인가봐. 다음 질문으로 넘어가는 걸로 양해해줄 수 있지? 고마워!")
        question = None
        continue

    print("\n1. 기억나")
    print("2. 잘 기억 안 나")
    print("3. 말하고 싶지 않아")
    print("4. 그만할래")
    user_choice = input("선택 (1/2/3/4): ")
  
    if user_choice == "4":
            print(f"{user_name}아, 오늘도 수고 많았어. 푹 쉬어!💚")
            break

    if user_choice == "1":
            user_answer = input(f"{user_name}의 답변: ")
            print("\n💛 감정을 선택해줘:")
            for idx, emo in enumerate(emotions):
                print(f"{idx+1}. {emo}")
            emotion_selected = emotions[int(input("번호 입력해줘: ")) - 1]
            emotion_stats[emotion_selected] += 1

            # 개인화된 다음 질문 생성
            question = generate_followup_question(user_answer)

    elif user_choice == "2":
        user_answer = "기억이 잘 나지 않아"
        emotion_selected = "잘 기억이 안 나"
        print("괜찮아! 흐릿한 기억도 있지🤗")
        question = None # 다음 질문은 일반 키워드로
            
    elif user_choice == "3":
        user_answer = "답변하고 싶지 않아"
        emotion_selected = "🔒말하고 싶지 않은 기억"
        print("물론! 편할 때 말해도 돼😊")
        question = None # 다음 질문은 일반 키워드로
            
    else:
        print("⚠️ 1,2,3 중에 다시 작성해줘😉")
        continue

    # 회상 기록 저장
    log_data = {
        "timestamp": datetime.now().isoformat(),
        "user": user_name,
        "category": category,
        "question": question,
        "ai_answer": ai_answer,
        "user_answer": user_answer,
        "emotion": emotion_selected
    }

    csv_path = "workspace/memory_log.csv"
    fieldnames = ["timestamp", "user", "category", "question", "ai_answer", "user_answer", "emotion"]

    try:
        with open(csv_path, "x", newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerow(log_data)
                
    except FileExistsError:
        with open(csv_path, "a", newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writerow(log_data)

    print("💚기록 저장 완료💚")

    # 질문 흐름 단계 이동은 초기 질문에서만 적용
    if question is None:
        current_stage = (current_stage + 1) % len(memory_flow)
        
# 감정 리포트 출력 
print("\n🧾 오늘 회상 감정 리포트 ")
for emo, count in emotion_stats.items():
    print(f"{emo}: {count}회")


🩷 질문: 오늘 저녁 기억나?하면 어떤 풍경이나 냄새가 떠올라?

🌿 나루 응답:
오늘 저녁 생각나? 그때의 풍경이나 냄새가 떠오르지? 나는 그때 정말 좋았던 걸 기억해. 마치 포근한 담요에 싸인 것처럼 따뜻했지. 지윤이는 어땠어?

1. 기억나
2. 잘 기억 안 나
3. 말하고 싶지 않아
4. 그만할래


선택 (1/2/3/4):  4


지윤아, 오늘도 수고 많았어. 푹 쉬어!💚

🧾 오늘 회상 감정 리포트 


In [11]:
# 감정 리포트 장기 저장 (그래프 생략 버전)
report_path = "workspace/emotion_report_log.csv"
now = datetime.now().strftime("%Y-%m-%d")

# 감정 통계 저장 준비
report_row = {"날짜": now}
report_row.update(emotion_stats)
fieldnames = ["날짜"] + emotions

try:
    with open(report_path, "x", newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerow(report_row)
except FileExistsError:
    with open(report_path, "a", newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writerow(report_row)

print("📁 오늘 감정 리포트가 emotion_report_log.csv에 저장됐어!")

📁 오늘 감정 리포트가 emotion_report_log.csv에 저장됐어!
